In [75]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from PIL import Image
import requests

In [76]:
option = webdriver.ChromeOptions()
option.add_argument("-incognito")
browser = webdriver.Chrome(executable_path=r'/home/srujandeshpande/chromedriver', options=option)

In [42]:
subscription_key = os.environ['OCR_SUBSCRIPTION_KEY']
endpoint = os.environ['OCR_ENDPOINT']

In [85]:
browser.get("https://vahan.nic.in/nrservices/faces/user/searchstatus.xhtml")

In [86]:
browser.save_screenshot("image.png") 

True

In [87]:
e = browser.find_element_by_xpath('//*[@id="capatcha"]/img')

location = dict(e.location)
size = dict(e.size)
print(location,size)

{'x': 766, 'y': 217} {'height': 30, 'width': 150}


In [88]:
im = Image.open("image.png") 
  
# Setting the points for cropped image 
left = location['x']
top = location['y']-10
right = location['x']+size['width']
bottom = location['y']+size['height']+10
  
# Cropped image of above dimensions 
im1 = im.crop((left, top, right, bottom)) 
im1.save("cropped.png")

In [89]:
ocr_url = endpoint + "vision/v3.0/read/analyze"

image_data = open("cropped.png", "rb").read()

headers = {'Ocp-Apim-Subscription-Key': subscription_key, 'Content-Type': 'application/octet-stream'}
params = {'language': 'en'}

response = requests.post(ocr_url, headers=headers, params=params, data=image_data)

response.raise_for_status()

In [90]:
result = dict()

analysis = {}
poll = True
while (poll):
    response_final = requests.get(
        response.headers["Operation-Location"], headers=headers)
    analysis = response_final.json()
    
    result = json.dumps(analysis, indent=2)

    time.sleep(1)
    if ("analyzeResult" in analysis):
        poll = False
    if ("status" in analysis and analysis['status'] == 'failed'):
        poll = False

print(result)

{
  "status": "succeeded",
  "createdDateTime": "2020-09-21T18:02:44Z",
  "lastUpdatedDateTime": "2020-09-21T18:02:44Z",
  "analyzeResult": {
    "version": "3.0.0",
    "readResults": [
      {
        "page": 1,
        "language": "en",
        "angle": 0,
        "width": 150,
        "height": 50,
        "unit": "pixel",
        "lines": [
          {
            "boundingBox": [
              8,
              18,
              136,
              18,
              136,
              32,
              8,
              32
            ],
            "text": "Greater No 3, 2",
            "words": [
              {
                "boundingBox": [
                  8,
                  20,
                  67,
                  20,
                  67,
                  32,
                  8,
                  32
                ],
                "text": "Greater",
                "confidence": 0.984
              },
              {
                "boundingBox": [
             

In [93]:
text = analysis['analyzeResult']['readResults'][0]['lines'][0]['text']

words = text.split()

answer = ""

if words[0]=='Greater':
    num1 = words[2][:-1]
    num2 = words[3]
    
    answer = num1 if num1 > num2 else num2
elif words[0]=='Smallest':
    num1 = words[2][:-1]
    num2 = words[3]
    
    answer = num1 if num1 < num2 else num2
elif words[1]=='+':
    num1 = int(words[0])
    num2 = int(words[2])

    answer = num1 + num2  
elif words[1]=='-':
    num1 = int(words[0])
    num2 = int(words[2])

    answer = num1 - num2  
elif words[1]=='*':
    num1 = int(words[0])
    num2 = int(words[2])

    answer = num1 * num2  
elif words[1]=='/':
    num1 = int(words[0])
    num2 = int(words[2])

    answer = num1 / num2  
answer = str(answer)
print(f"{text}: {answer}")

Greater No 3, 2: 3
